# Анализ книжного рынка (SQL)

## Цели и задачи исследования

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Основная цель исследования — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Исследование будет состоять из нескольких задач-пунктов:
- Посчитайть, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайть количество обзоров и среднюю оценку;
- Определитб издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных**

В нашем распоряжении 5 таблиц:

`books` - содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

`authors` - содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

`publishers` - содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

`ratings` - содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

`reviews` - содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Знакомство с данными

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

Первым делом посмотрим на содержание всех таблиц.

In [3]:
query = '''SELECT * FROM books LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''SELECT * FROM authors LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''SELECT * FROM ratings LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''SELECT * FROM reviews LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Исследовательский анализ

### Задача 1
Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
query = '''SELECT COUNT(*)
           FROM books
           WHERE publication_date > '2000-01-01' '''

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


В нашей базе хранится информация о 819 книгах, которые были опубликованы после 1 января 2000 года.

### Задание 2
Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
query = '''SELECT b.book_id, b.title, COUNT(DISTINCT re.review_id) review_count, ROUND(AVG(ra.rating), 2) raiting
           FROM books b
           LEFT JOIN ratings ra on b.book_id=ra.book_id
           LEFT JOIN reviews re on b.book_id=re.book_id
           GROUP BY 1
           ORDER BY 3 DESC, 4 DESC'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_count,raiting
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


В нашей базе хранится 1000 книг на которые сделали обзор, причем число обзоров на одну книгу имеет разброс от 0 до 7. Рейтинги книг при этом также могут быть самыми разнообразными и не зависят от числа отзывов: книга может иметь множество отзывов, но низкий рейтинг и наоборот.

### Задание 3
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query = '''SELECT p.publisher_id, p.publisher, COUNT(DISTINCT b.book_id)
           FROM publishers p
           JOIN books b on b.publisher_id=p.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher_id
           ORDER BY COUNT(b.book_id) DESC
           LIMIT 1'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count
0,212,Penguin Books,42


Самое продуктивное издательство по числу выпущенных книг (произведений толще 50 стр.) Penguin Books - оно выпустило 42 произведения.

### Задание 4
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query = '''WITH pop_books AS (
           SELECT b.book_id, b.author_id, AVG(r.rating) raiting
           FROM books b
           JOIN ratings r on b.book_id=r.book_id
           GROUP BY b.book_id
           HAVING COUNT(r.rating_id) >= 50
           )
           SELECT a.author_id, a.author, ROUND(AVG(raiting), 2) raiting
           FROM authors a
           JOIN pop_books pb on pb.author_id=a.author_id
           GROUP BY a.author_id
           ORDER BY ROUND(AVG(raiting), 2) DESC
           LIMIT 1
           '''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,raiting
0,236,J.K. Rowling/Mary GrandPré,4.28


Автор с самым высоким рейтингом книг, которые имеют более 50 оценок, - Джоан Роулинг.

### Задание 5
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query = '''WITH super_rev AS (
           SELECT ra.username, COUNT(DISTINCT ra.rating_id) score_count, COUNT(DISTINCT re.review_id) review_count
           FROM ratings ra
           FULL JOIN reviews re on ra.username=re.username
           GROUP BY ra.username
           HAVING COUNT(DISTINCT ra.rating_id) > 48
           )
           SELECT AVG(review_count)
           FROM super_rev'''

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров у пользователей, которые поставили больше 48 оценок, - 24.

## Выводы

Проведя анализ данных сервиса для книг, получили следующие результаты:
 - В нашей базе хранится информация о 819 книгах, которые были опубликованы после 1 января 2000 года
 - В нашей базе хранится 1000 книг на которые сделали обзор, причем число обзоров на одну книгу имеет разброс от 0 до 7. Рейтинги книг при этом также могут быть самыми разнообразными и не зависят от числа отзывов: книга может иметь множество отзывов, но низкий рейтинг и наоборот
 - Самое продуктивное издательство по числу выпущенных книг (произведений толще 50 стр.) Penguin Books - оно выпустило 42 произведения
 - Автор с самым высоким рейтингом книг, которые имеют более 50 оценок, - Джоан Роулинг
 - Среднее количество обзоров у пользователей, которые поставили больше 48 оценок, - 24